In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genre-df/sample_submission.csv
/kaggle/input/genre-df/train.csv
/kaggle/input/genre-df/test.csv
/kaggle/input/distilroberta/distilroberta_test.npy
/kaggle/input/distilroberta/distilroberta_df.npy


In [2]:
# Reading the df
df = pd.read_csv('/kaggle/input/genre-df/train.csv')
test = pd.read_csv('/kaggle/input/genre-df/test.csv') 
sample_submission = pd.read_csv('/kaggle/input/genre-df/sample_submission.csv')
classes_list = sample_submission.columns[1:]
print(len(classes_list))
print(df.shape)
print(test.shape)
print(sample_submission.shape)
df.head()

70
(3000, 3)
(864, 2)
(864, 71)


,index,plot_synopsis,tags
0,0,Charlyne Yi embarks on a quest across America ...,"paranormal, psychedelic, storytelling, home movie"
1,1,The movie is a parody of classic kung fu movie...,"cult, comedy, comic, violence, flashback"
2,2,"In 2011, João ""Zero"", a bitter but brilliant p...",romantic
3,3,Kent McCoy is a loving husband and father who ...,murder
4,4,The animated introduction revolves around how ...,"comedy, murder, violence, horror, flashback, s..."


In [3]:
import re
import nltk
from nltk.corpus import stopwords

# Download NLTK stopwords data
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Cleaning plot_synopsis
import re
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')
nlp = spacy.load('en_core_web_sm')

def clean_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ", text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    
    # Lemmatization using spaCy
    doc = nlp(" ".join(stemmed_tokens))
    lemmatized_tokens = [token.lemma_ for token in doc]
    
    # Joining tokens back to text
    text = ' '.join(lemmatized_tokens)
    
    print('done')
    return text


# Converting tags to list of tags
def conv_tags(text):
    words = [word.strip().replace(' ', '_') for word in text.split(',')]
    return words
df['cleaned_plot_synopsis'] = df['plot_synopsis'].apply(lambda x: clean_text(x))
test['cleaned_plot_synopsis'] = test['plot_synopsis'].apply(lambda x: clean_text(x))
df['cleaned_tags'] = df['tags'].apply(lambda x: conv_tags(x))
# print(test.head())
df.head()

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
d

,index,plot_synopsis,tags,cleaned_plot_synopsis,cleaned_tags
0,0,Charlyne Yi embarks on a quest across America ...,"paranormal, psychedelic, storytelling, home movie",charlyn yi embark quest across america make do...,"[paranormal, psychedelic, storytelling, home_m..."
1,1,The movie is a parody of classic kung fu movie...,"cult, comedy, comic, violence, flashback",movi parodi classic kung fu movi featur footag...,"[cult, comedy, comic, violence, flashback]"
2,2,"In 2011, João ""Zero"", a bitter but brilliant p...",romantic,jo zero bitter brilliant physicist spend day b...,[romantic]
3,3,Kent McCoy is a loving husband and father who ...,murder,kent mccoy love husband father host birthday p...,[murder]
4,4,The animated introduction revolves around how ...,"comedy, murder, violence, horror, flashback, s...",anim introduct revolv around devon voic snoop ...,"[comedy, murder, violence, horror, flashback, ..."


In [4]:
df.to_csv('new_train.csv',index=False)
test.to_csv('new_test.csv',index=False)